<a href="https://colab.research.google.com/github/ktynski/Marketing_Automations_Notebooks_With_GPT/blob/main/Automatic_Question_Expander_ala_AnswerThePublic_(Public).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
import openai
import pandas as pd
import concurrent.futures

openai.api_key = "your Openai api key"



def generate_text(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.6,
    )
    return response.choices[0].text

def generate_subtopics(topic):
    prompt = f"Generate 40 common subtopics related to {topic}"
    subtopics = generate_text(prompt).strip().split("\n")
    return subtopics

def generate_subsubtopics(subtopic):
    prompt = f"Generate 20 common subsubtopics related to {subtopic}"
    subsubtopics = generate_text(prompt).strip().split("\n")
    print(subsubtopics)
    return subsubtopics

def generate_questions(topic, subtopic, subsubtopic):
    prompt = f"""You are an all knowing AI that specializes in search behavior, SEO, and search intent.
    You are especially good at predicting questions related to topics.
    Please Generate 30 of the most common questions related to {subsubtopic}
    within the context of {subtopic} in the larger topic of {topic}"""
    questions = generate_text(prompt).strip().split("\n")
    return [question.split(". ")[-1] for question in questions]

def process_subsubtopic(topic, subtopic, subsubtopic):
    questions = generate_questions(topic, subtopic, subsubtopic)
    data = {
        "topic": topic,
        "subtopic": subtopic,
        "subsubtopic": subsubtopic,
    }
    for i, question in enumerate(questions):
        data[f"Question {i + 1}"] = question
    return data

def main(topic):
    subtopics = generate_subtopics(topic)
    data = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for subtopic in subtopics:
            subsubtopics = generate_subsubtopics(subtopic)
            results = [executor.submit(process_subsubtopic, topic, subtopic, subsubtopic) for subsubtopic in subsubtopics]
            for f in concurrent.futures.as_completed(results):
                data.append(f.result())
    df = pd.DataFrame(data)
    return df

if __name__ == "__main__":
    topic = "Home Security"
    df = main(topic)
    print(df)


In [ ]:
df.to_csv("homesecurityquestions2.csv")